In [4]:
# %load ../python_private/imports.py
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_columns', 35)
#the lines below get rid of the plotting library warnings
import warnings
warnings.filterwarnings("ignore")


In [5]:
raw=pd.read_csv('data.csv')
print(raw.shape)
raw.head()

(54712, 1002)


,user_id,0,1,2,3,4,5,6,7,8,...,991,992,993,994,995,996,997,998,999,label
0,cb8d2cfc-eecb-11e9-b094-8c8590a62204,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,cba33772-eecb-11e9-b094-8c8590a62204,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,89102,0,1
2,cb934b50-eecb-11e9-b094-8c8590a62204,0,0,1795,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,cb9261cc-eecb-11e9-b094-8c8590a62204,0,0,0,0,0,0,0,104,0,...,0,0,0,0,0,0,0,0,0,0
4,cb94cfe8-eecb-11e9-b094-8c8590a62204,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,1


In [7]:
raw.dtypes

user_id    object
0           int64
1           int64
2           int64
3           int64
            ...  
996         int64
997         int64
998         int64
999         int64
label       int64
Length: 1002, dtype: object

In [19]:
labels=pd.read_csv('labels.csv')
print(labels.shape)
labels.head()

(54711, 1)


,0
0,1
1,1
2,0
3,1
4,1


* notice that `data.csv` has one more row than `labels.csv`. It seems that the first row in `data.csv` is the missing one, and otherwise all the information contained in `labels.csv` is redundant:

In [20]:
sum(raw.loc[1:,'label'].values-labels['0'].values)

0

In [21]:
ids=pd.read_csv('user_ids.csv')
print(ids.shape)
ids.head()

(54712, 1)


,user_id
0,cb8d2cfc-eecb-11e9-b094-8c8590a62204
1,cba33772-eecb-11e9-b094-8c8590a62204
2,cb934b50-eecb-11e9-b094-8c8590a62204
3,cb9261cc-eecb-11e9-b094-8c8590a62204
4,cb94cfe8-eecb-11e9-b094-8c8590a62204


* It seems that all the information in `user_ui.csv` corresponds to the first column of `data.csv` and therefore the former file is also redundant:

In [22]:
sum(ids.user_id == raw.user_id)

54712